# Two Body Problem

So now we're going to model a two-body gravitational force problem by splitting the problem into two one-body problems for the separation and center of mass vectors. First, some theory:

### Getting the Equations of Motion

We can get the equations of motion $\ddot x_1, \ddot y_1, \ddot x_2, \ddot y_2$ two different ways: first, by geometric arguments using forces (a la 1250/2300), or via the Euler Lagrange equations. The E-L equations would simplify the problem fairly well using polar coordinates, but in Cartesian coordinates, the Newton picture works simply enough (though we will utilize *both* methods in order to error-check our work).

The acceleration of each coordinate, then, is the net force in that coordinate's direction divided by the mass of the accelerating body;

$
\begin{align}
\ddot m_1 x_1 &= \mathbf{F_{12}} \cdot \hat{x}
\end{align}
$

and similar the same for the other generalized coordinates.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
# for animations
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
class TwoBodyGrav:
    
    def __init__(self, G, m1, m2):
        """ Initializes class members
        
            Parameters:
                G - gravitation constant
                m1, m2 - masses of bodies
        """
        self.G = G
        self.m1 = m1
        self.m2 = m2
        
    def dy_dt(self, t, y):
        """ Calculates derivative of state vector 
            [dx1, ddx1, dy1, ddy1, dx2, ddx2, dy2, ddy2]
        
            Parameters:
                y - state vector [x1, dx1, y1, dy1, x2, dx2, y2, dy2]
                t - time        
        """
        k = G/(((y[0]-y[4])**2.+(y[2]-y[6])**2.)**1.5)
        
        ddx1 = -k*m2*(y[0]-y[4])
        ddx2 = k*m1*(y[0]-y[4])
        ddy1 = -k*m2*(y[2]-y[6])
        ddy2 = k*m1*(y[2]-y[6])
        return [y[1], ddx1, y[3], ddy1, y[5], ddx2, y[7], ddy2 ]
    
    def solve_ode(self, t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                  x2_0, x2_dot_0, y2_0, y2_dot_0, abserr=1.0e-9, relerr=1.0e-9):
        """ Solves the ODE defined by dy_dt using scipy solve_ivp and
            the given initial conditions
            
            Parameters:
                t_pts - discretized time grid
                xn, yn, etc - initial positions/velocities
                abserr/relerr - absolute and relative error
        """
        
        y = [x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0]
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), y, t_eval=t_pts,
                            atol=abserr, rtol=relerr,  method='RK45')
        x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot = solution.y
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot
    
    def solve_ode_Leapfrog(self, t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                  x2_0, x2_dot_0, y2_0, y2_dot_0):
        """
            Solve the ODE given initial conditions with the Leapfrog (Verlet) method.
        
            Paramters:
                See solve_ode.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, r_dot_half, phi with zeros
        num_t_pts = len(t_pts)
        x1 = np.zeros(num_t_pts)
        x1_dot = np.zeros(num_t_pts)
        x1_dot_half = np.zeros(num_t_pts)
        
        y1 = np.zeros(num_t_pts)
        y1_dot = np.zeros(num_t_pts)
        y1_dot_half = np.zeros(num_t_pts)
        
        x2 = np.zeros(num_t_pts)
        x2_dot = np.zeros(num_t_pts)
        x2_dot_half = np.zeros(num_t_pts)
        
        y2 = np.zeros(num_t_pts)
        y2_dot = np.zeros(num_t_pts)
        y2_dot_half = np.zeros(num_t_pts)
        
        # initial conditions
        x1[0] = x1_0
        x1_dot[0] = x1_dot_0
        
        y1[0] = y1_0
        y1_dot[0] = y1_dot_0
        
        x2[0] = x2_0
        x2_dot[0] = x2_dot_0
        
        y2[0] = y2_0
        y2_dot[0] = y2_dot_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            
            # first, set state vector
            y = [x1[i], x1_dot[i], y1[i], y1_dot[i], x2[i], x2_dot[i], y2[i], y2_dot[i]]
            
            # now, kick the ENTIRE VECTOR every time, or else we'll be drifting with outdated states
            x1_dot_half[i] = x1_dot[i] + self.dy_dt(t,y)[1] * delta_t/2.
            y1_dot_half[i] = y1_dot[i] + self.dy_dt(t,y)[3] * delta_t/2.
            x2_dot_half[i] = x2_dot[i] + self.dy_dt(t,y)[5] * delta_t/2.
            y2_dot_half[i] = y2_dot[i] + self.dy_dt(t,y)[7] * delta_t/2.
            
            # now drift (again with everything)
            x1[i+1] = x1[i] + x1_dot_half[i] * delta_t
            y1[i+1] = y1[i] + y1_dot_half[i] * delta_t 
            x2[i+1] = x2[i] + x2_dot_half[i] * delta_t 
            y2[i+1] = y2[i] + y2_dot_half[i] * delta_t 
            
            # update our positions
            y = [x1[i+1], x1_dot[i], y1[i+1], y1_dot[i], x2[i+1], x2_dot[i], y2[i+1], y2_dot[i]]
            
            # and kick again!
            x1_dot[i+1] = x1_dot_half[i] + self.dy_dt(t,y)[1] * delta_t/2.
            y1_dot[i+1] = y1_dot_half[i] + self.dy_dt(t,y)[3] * delta_t/2.
            x2_dot[i+1] = x2_dot_half[i] + self.dy_dt(t,y)[5] * delta_t/2.
            y2_dot[i+1] = y2_dot_half[i] + self.dy_dt(t,y)[7] * delta_t/2.
            
            # phi[i+1] = phi[i] + self.dy_dt(t,y)[2] * delta_t
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot 
        

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

In [ ]:
# Labels for individual plot axes
r_vs_time_labels = (r'$x$', r'$y$')
# r_dot_vs_time_labels = (r'$x$', r'$dr/dt(t)$')
# state_space_labels = (r'$r$', r'$dr/dt$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 50
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)

G = 1
m1 = 1
m2 = 1

# Instantiate two-body gravitation class 
G1 = TwoBodyGrav(G=G, m1=m1, m2=m2)

In [ ]:
x1_0 = -1
x1_dot_0 = 0.
y1_0 = 0.
y1_dot_0 = 0.4

x2_0 = 1
x2_dot_0 = 0.
y2_0 = 0.
y2_dot_0 = -0.5

x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot = G1.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0 )

fig = plt.figure(figsize=(8,8))
ax_a = fig.add_subplot(111)

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(x1[start : stop], y1[start : stop], 
            axis_labels=r_vs_time_labels, 
            color='blue',
            label='Body 1', 
            title=r'$Two-Body Problem',
            ax=ax_a)
            
plot_y_vs_x(x2[start : stop], y2[start : stop], 
            axis_labels=r_vs_time_labels, 
            color='red',
            label=r'Body 2', 
            title=r'Two-Body Problem', 
            ax=ax_a)

fig.tight_layout()

This graph makes sense: we know analytically in a two-body problem that the center of mass moves with constant velocity, and that the two bodies each orbit around this center, which is what appears in the image.

# What happens if one mass is much larger?

First, let's show the equations of motion for each component:


$\begin{align}
 \ddot x_1 &= -\frac{Gm_2(x_1-x_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{\frac{3}{2}}}\\
 \ddot x_2 &= \frac{Gm_1(x_1-x_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{\frac{3}{2}}}\\
  \ddot y_1 &= -\frac{Gm_2(y_1-y_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{\frac{3}{2}}}\\
 \ddot y_2 &= \frac{Gm_1(y_1-y_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{\frac{3}{2}}}
\end{align}$

If $m_1$ is much larger than $m_2$, it follows that the accelerations of $x_1$ and $y_1$ are much smaller than those of $x_2$ and $y_2$, so we can consider them negligible. Thus, the body with mass $m_1$ is effectively moving with constant velocity, and we've reduced the problem to a one-body central force problem for the relative motion (the motion of $m_2$). We can see that in our above graph, where 

In [ ]:
# Now, leapfroggin'!
lx1, lx1_dot, ly1, ly1_dot, lx2, lx2_dot, ly2, ly2_dot = G1.solve_ode_Leapfrog(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0 )

fig = plt.figure(figsize=(8,8))
ax_a = fig.add_subplot(1,1,1)

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(x1[start : stop], y1[start : stop], 
            axis_labels=r_vs_time_labels, 
            color='blue',
            label='Body 1', 
            title=r'$Verlet Two-Body Problem',
            ax=ax_a)
            
plot_y_vs_x(x2[start : stop], y2[start : stop], 
            axis_labels=r_vs_time_labels, 
            color='red',
            label=r'Body 2', 
            title=r'Verlet Two-Body Problem', 
            ax=ax_a)
fig.tight_layout()

Looks pretty similar. We wouldn't expect too much of a difference in a short span of time.

In [ ]:
# Now, check energies:
# scalar initial energy
E0 = (m1*(x1_dot_0**2 + y1_dot_0**2)+m2*(x2_dot_0**2+y2_dot_0**2))/2. - G*m1*m2/((x1_0-x2_0)**2+(y1_0-y2_0)**2)**(3./2.)
# energies sampled along discretized time domain
E_rk4 = (m1*(x1_dot**2 + y1_dot**2)+m2*(x2_dot**2+y2_dot**2))/2. - G*m1*m2/((x1-x2)**2+(y1-y2)**2)**(3./2.)
E_lf = (m1*(lx1_dot**2 + ly1_dot**2)+m2*(lx2_dot**2+ly2_dot**2))/2. - G*m1*m2/((lx1-lx2)**2+(ly1-ly2)**2)**(3./2.)

# Let's hope we're below 10% error!
rk4_rel_dif = np.abs((E_rk4[0]-E_rk4)/E_rk4[0])
lf_rel_dif = np.abs((E_lf[0]-E_lf)/E_lf[0])

# Let's plot the relative errors to see...
ERel_fig = plt.figure(figsize=(5,5))
ERel_ax_a = ERel_fig.add_subplot(1,1,1)

plot_y_vs_x(t_pts[start : stop], rk4_rel_dif[start : stop], 
            color='blue',
            label=r'RK4', 
            title=r'Relative Energy Discrepancy',
            ax=ERel_ax_a)
plot_y_vs_x(t_pts[start : stop], lf_rel_dif[start : stop], 
            color='red',
            label=r'Leapfrog', 
            title=r'Relative Energy Discrepancy',
            ax=ERel_ax_a)

Now this is kind of odd; both the RK4 and Leapfrog (Verlet) integrators seem to have cyclic discrepancies in energy, which doesn't appear right; RK4 should accrue some degree of energy drift over time. The error also jumps up to over 40%! Perhaps we have a flawed implementation of the two-body problem solution, or multiple errors that still cause the integration methods to agree...

In [ ]:
E_fig = plt.figure(figsize=(10,5))
E_ax_a = E_fig.add_subplot(1,1,1)

plot_y_vs_x(t_pts[start : stop], E_rk4[start : stop], 
            color='blue',
            label=r'RK4', 
            title=r'Total Energy',
            ax=E_ax_a)
plot_y_vs_x(t_pts[start : stop], E_lf[start : stop], 
            color='red',
            label=r'Leapfrog', 
            title=r'Total Energy',
            ax=E_ax_a)

Let's try a widget before an animation.

In [ ]:
# Import explicitly the widgets we might use (add more as needed!) 
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Tab, Label, Checkbox
from ipywidgets import FloatSlider, IntSlider, Play, Dropdown, HTMLMath 

from IPython.display import display

In [ ]:
def update_plot(G, m1, m2, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0):
    # Labels for individual plot axes
    phi_vs_time_labels = (r'$t$', r'$\phi(t)$')
    phi_dot_vs_time_labels = (r'$t$', r'$d\phi/dt(t)$')
    state_space_labels = (r'$\phi$', r'$d\phi/dt$')

    # Common plotting time (generate the full time then use slices)
    t_start = 0.
    t_end = 50
    delta_t = 0.05
    
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)
    
    # Instantiate a pendulum 
    G1 = TwoBodyGrav(G=G, m1=m1, m2=m2)
    
    x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, = G1.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0)
    
    # start the plot!
    fig = plt.figure(figsize=(8,8))
    overall_title = 'Two Body Problem:  ' + \
                rf'  $x_1(0) = {x1_0:.2f},$' + \
                rf' $\dot x_1(0) = {x1_dot_0:.2f}$' + \
                rf'  $y_1(0) = {y1_0:.2f},$' + \
                rf' $\dot y_1(0) = {y1_dot_0:.2f}$' + \
                rf'  $x_2(0) = {x2_0:.2f},$' + \
                rf' $\dot x_2(0) = {x2_dot_0:.2f}$' + \
                rf'  $y_2(0) = {y2_0:.2f},$' + \
                rf' $\dot y_2(0) = {y2_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
    fig.suptitle(overall_title, va='baseline')
    
    # first plot: phi plot 
    ax_a = fig.add_subplot(1,1,1)                  

    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(x1[start : stop], y1[start : stop], 
                axis_labels=r_vs_time_labels, 
                color='blue',
                label='Body 1', 
                title=r'$Two-Body Problem',
                ax=ax_a)
            
    plot_y_vs_x(x2[start : stop], y2[start : stop], 
                axis_labels=r_vs_time_labels, 
                color='red',
                label=r'Body 2', 
                title=r'Two-Body Problem', 
                ax=ax_a)

# We need control sliders for each variable
slider_G = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$G$ :')
slider_m1 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_1$ :')
slider_m2 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_2$ :')
slider_x1_0 = FloatSlider(value=-1, min=-5, max=5, step=0.1,
                      description=r'$x_1(0)$ :')
slider_dx1_0 = FloatSlider(value=0.0, min=-5, max=5., step=0.1,
                      description=r'$\dot x_1(0)$ :')
slider_y1_0 = FloatSlider(value=0, min=-5, max=5, step=0.1,
                      description=r'$y_1(0)$ :')
slider_dy1_0 = FloatSlider(value=0.4, min=-5, max=5., step=0.1,
                      description=r'$\dot y_1(0)$ :')
slider_x2_0 = FloatSlider(value=1, min=-5, max=5, step=0.1,
                      description=r'$x_2(0)$ :')
slider_dx2_0 = FloatSlider(value=0.0, min=-5, max=5., step=0.1,
                      description=r'$\dot x_2(0)$ :')
slider_y2_0 = FloatSlider(value=0, min=-5, max=5, step=0.1,
                      description=r'$y_2(0)$ :')
slider_dy2_0 = FloatSlider(value=-0.4, min=-5, max=5., step=0.1,
                      description=r'$\dot y_2(0)$ :')

# And now we plot the output with our dictionary of variable:slider correpondences
plot_out = widgets.interactive_output(update_plot,
                                      dict(G=slider_G, m1=slider_m1, m2=slider_m2,
                                           x1_0=slider_x1_0, x2_0=slider_x2_0,
                                           x1_dot_0=slider_dx1_0, x2_dot_0=slider_dx2_0,
                                           y1_0=slider_y1_0, y2_0=slider_y2_0,
                                           y1_dot_0=slider_dy1_0, y2_dot_0=slider_dy2_0
                                          )
                                     )

In [ ]:
UI_box = VBox([slider_G, slider_m1, slider_m2,
                slider_x1_0, slider_x2_0,
                slider_dx1_0, slider_dx2_0,
                slider_y1_0, slider_y2_0,
                slider_dy1_0, slider_dy2_0, plot_out])
display(UI_box)

In [ ]:
%%capture

x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, = G1.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0)

fig_anim1 = plt.figure(figsize=(5,5))
ax_anim1 = fig_anim1.add_subplot(111,  xlim=(-5,5), ylim=(-5,5))
ax_anim1.set_aspect('equal')
ax_anim1.grid()
line_anim1, = ax_anim1.plot(x1[0], y1[0], 'bo')

def animate_double(i):
    xdata = [x1[i], x2[i]]
    ydata = [y1[i], y2[i]]
    
    line_anim1.set_data(xdata,ydata)
    return line_anim1,

anim1 = animation.FuncAnimation(fig_anim1, animate_double, init_func=None, frames=400, interval=10, blit=True, repeat=False)

In [ ]:
HTML(anim1.to_jshtml())

This looks nice, actually! It's easy to look at the center of mass momentum and compare it to the initial conditions.

In [ ]:
class ThreeBodyGrav:
    
    def __init__(self, G, m1, m2, m3):
        """ Initializes class members
        
            Parameters:
                G - gravitation constant
                m1, m2, m3 - masses of bodies
        """
        self.G = G
        self.m1 = m1
        self.m2 = m2
        self.m3 = m3
        
    def dy_dt(self, t, y):
        """ Calculates derivative of state vector 
            [dx1, ddx1, dy1, ddy1, dx2, ddx2, dy2, ddy2, dx3, ddx3, dy3, ddx3]
        
            Parameters:
                y - state vector [x1, dx1, y1, dy1, x2, dx2, y2, dy2, x3, dx3, y3, dy3]
                t - time        
        """
        # We just need to account for all the pair interactions
        k12 = G/(((y[0]-y[4])**2.+(y[2]-y[6])**2.)**1.5)
        k23 = G/(((y[4]-y[8])**2.+(y[6]-y[10])**2.)**1.5)
        k13 = G/(((y[0]-y[8])**2.+(y[2]-y[10])**2.)**1.5)
        
        # So the acceleration of x1 is due to 1-2 and 1-3 interactions
        ddx1 = -k12*m2*(y[0]-y[4])-k13*m3*(y[0]-y[8])
        ddy1 = -k12*m2*(y[2]-y[6])-k13*m3*(y[2]-y[10])
        # Now 1-2 and 2-3
        ddx2 = k12*m1*(y[0]-y[4])-k23*m3*(y[4]-y[8])
        ddy2 = k12*m1*(y[2]-y[6])-k23*m3*(y[6]-y[10])
        # Now 1-3 and 2-3 
        ddx3 = k23*m2*(y[4]-y[8])+k13*m1*(y[0]-y[8])
        ddy3 = k23*m2*(y[6]-y[10])+k13*m1*(y[2]-y[10])
        return [y[1], ddx1, y[3], ddy1, y[5], ddx2, y[7], ddy2, y[9], ddx3, y[11], ddy3]
    
    def solve_ode(self, t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                  x2_0, x2_dot_0, y2_0, y2_dot_0, x3_0, x3_dot_0,
                  y3_0, y3_dot_0, abserr=1.0e-9, relerr=1.0e-9):
        """ Solves the ODE defined by dy_dt using scipy solve_ivp and
            the given initial conditions
            
            Parameters:
                t_pts - discretized time grid
                xn, yn, etc - initial positions/velocities
                abserr/relerr - absolute and relative error
        """
        
        y = [x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0, x3_0, x3_dot_0, y3_0, y3_dot_0]
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), y, t_eval=t_pts,
                            atol=abserr, rtol=relerr,  method='RK45')
        x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, x3, x3_dot, y3, y3_dot = solution.y
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, x3, x3_dot, y3, y3_dot
    
    def solve_ode_Leapfrog(self, t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                  x2_0, x2_dot_0, y2_0, y2_dot_0, x3_0, x3_dot_0, y3_0, y3_dot_0):
        """
            Solve the ODE given initial conditions with the Leapfrog (Verlet) method.
        
            Paramters:
                See solve_ode.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, r_dot_half, phi with zeros
        num_t_pts = len(t_pts)
        x1 = np.zeros(num_t_pts)
        x1_dot = np.zeros(num_t_pts)
        x1_dot_half = np.zeros(num_t_pts)
        
        y1 = np.zeros(num_t_pts)
        y1_dot = np.zeros(num_t_pts)
        y1_dot_half = np.zeros(num_t_pts)
        
        x2 = np.zeros(num_t_pts)
        x2_dot = np.zeros(num_t_pts)
        x2_dot_half = np.zeros(num_t_pts)
        
        y2 = np.zeros(num_t_pts)
        y2_dot = np.zeros(num_t_pts)
        y2_dot_half = np.zeros(num_t_pts)
        
        x3 = np.zeros(num_t_pts)
        x3_dot = np.zeros(num_t_pts)
        x3_dot_half = np.zeros(num_t_pts)
        
        y3 = np.zeros(num_t_pts)
        y3_dot = np.zeros(num_t_pts)
        y3_dot_half = np.zeros(num_t_pts)
        
        # initial conditions
        x1[0] = x1_0
        x1_dot[0] = x1_dot_0
        
        y1[0] = y1_0
        y1_dot[0] = y1_dot_0
        
        x2[0] = x2_0
        x2_dot[0] = x2_dot_0
        
        y2[0] = y2_0
        y2_dot[0] = y2_dot_0
        
        x3[0] = x3_0
        x3_dot[0] = x3_dot_0
        
        y3[0] = y3_0
        y3_dot[0] = y3_dot_0

        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            
            # first, set state vector
            y = [x1[i], x1_dot[i], y1[i], y1_dot[i], x2[i], x2_dot[i], y2[i], y2_dot[i], x3[i], x3_dot[i], y3[i], y3_dot[i]]
            
            # now, kick the ENTIRE VECTOR every time, or else we'll be drifting with outdated states
            x1_dot_half[i] = x1_dot[i] + self.dy_dt(t,y)[1] * delta_t/2.
            y1_dot_half[i] = y1_dot[i] + self.dy_dt(t,y)[3] * delta_t/2.
            x2_dot_half[i] = x2_dot[i] + self.dy_dt(t,y)[5] * delta_t/2.
            y2_dot_half[i] = y2_dot[i] + self.dy_dt(t,y)[7] * delta_t/2.
            x3_dot_half[i] = x3_dot[i] + self.dy_dt(t,y)[9] * delta_t/2.
            y3_dot_half[i] = y3_dot[i] + self.dy_dt(t,y)[11] * delta_t/2.
            
            # now drift (again with everything)
            x1[i+1] = x1[i] + x1_dot_half[i] * delta_t
            y1[i+1] = y1[i] + y1_dot_half[i] * delta_t 
            x2[i+1] = x2[i] + x2_dot_half[i] * delta_t 
            y2[i+1] = y2[i] + y2_dot_half[i] * delta_t 
            x3[i+1] = x3[i] + x3_dot_half[i] * delta_t 
            y3[i+1] = y3[i] + y3_dot_half[i] * delta_t 
            
            # update our positions
            y = [x1[i+1], x1_dot[i], y1[i+1], y1_dot[i], x2[i+1], x2_dot[i], y2[i+1], y2_dot[i], x3[i+1], x3_dot[i], y3[i+1], y3_dot[i]]
            
            # and kick again!
            x1_dot[i+1] = x1_dot_half[i] + self.dy_dt(t,y)[1] * delta_t/2.
            y1_dot[i+1] = y1_dot_half[i] + self.dy_dt(t,y)[3] * delta_t/2.
            x2_dot[i+1] = x2_dot_half[i] + self.dy_dt(t,y)[5] * delta_t/2.
            y2_dot[i+1] = y2_dot_half[i] + self.dy_dt(t,y)[7] * delta_t/2.
            x3_dot[i+1] = x3_dot_half[i] + self.dy_dt(t,y)[9] * delta_t/2.
            y3_dot[i+1] = y3_dot_half[i] + self.dy_dt(t,y)[11] * delta_t/2.
            
            # phi[i+1] = phi[i] + self.dy_dt(t,y)[2] * delta_t
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, x3, x3_dot, y3, y3_dot
        

In [ ]:
# Labels for individual plot axes
r_vs_time_labels = (r'$x$', r'$y$')
# r_dot_vs_time_labels = (r'$x$', r'$dr/dt(t)$')
# state_space_labels = (r'$r$', r'$dr/dt$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 4.2
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  # should be  

G = 1
m1 = 1
m2 = 1
m3 = 1

# Instantiate a pendulum 
G2 = ThreeBodyGrav(G=G, m1=m1, m2=m2, m3=m3)

In [ ]:
x1_0 = -1
x1_dot_0 = 0.
y1_0 = 0.
y1_dot_0 = 0.5

x2_0 = 0.5
x2_dot_0 = np.sqrt(3.)/4.
y2_0 = np.sqrt(3.)/2.
y2_dot_0 = -0.25

x3_0 = 0.5
x3_dot_0 = -np.sqrt(3)/4.
y3_0 = -np.sqrt(3.)/2.
y3_dot_0 = -0.25

x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, x3, x3_dot, y3, y3_dot = G2.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0, x3_0, x3_dot_0, y3_0, y3_dot_0)

fig = plt.figure(figsize=(8,8))
ax_a = fig.add_subplot(111)

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(x1[start : stop], y1[start : stop], 
            axis_labels=r_vs_time_labels, 
            color='blue',
            label='Body 1', 
            title=r'$Three-Body Problem',
            ax=ax_a)
            
plot_y_vs_x(x2[start : stop], y2[start : stop], 
            axis_labels=r_vs_time_labels, 
            color='red',
            label=r'Body 2', 
            title=r'Three-Body Problem', 
            ax=ax_a)

plot_y_vs_x(x3[start : stop], y3[start : stop], 
            axis_labels=r_vs_time_labels, 
            color='green',
            label=r'Body 3',
            title=r'Three-Body Problem', 
            ax=ax_a)

fig.tight_layout()

The initial conditions we're using have a rotationally symmetric nature, and so we'd expect each orbit to be identical, as the graph seems to indicate. Being able to animate this graph would make for a pretty good visualization, tool. But let's start with a widget.

In [ ]:
def update_3body_plot(G, m1, m2, m3, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0, x3_0, x3_dot_0, y3_0, y3_dot_0):

    # Common plotting time (generate the full time then use slices)
    t_start = 0.
    t_end = 25
    delta_t = 0.05
    
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)
    
    # Instantiate a pendulum 
    G2 = ThreeBodyGrav(G=G, m1=m1, m2=m2, m3=m3)
    
    x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, x3, x3_dot, y3, y3_dot = G2.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0, x3_0, x3_dot_0, y3_0, y3_dot_0)
    
    # start the plot!
    fig = plt.figure(figsize=(8,8))
    overall_title = 'Two Body Problem:  ' + \
                rf'  $x_1(0) = {x1_0:.2f},$' + \
                rf' $\dot x_1(0) = {x1_dot_0:.2f}$' + \
                rf'  $y_1(0) = {y1_0:.2f},$' + \
                rf' $\dot y_1(0) = {y1_dot_0:.2f}$' + \
                rf'  $x_2(0) = {x2_0:.2f},$' + \
                rf' $\dot x_2(0) = {x2_dot_0:.2f}$' + \
                rf'  $y_2(0) = {y2_0:.2f},$' + \
                rf' $\dot y_2(0) = {y2_dot_0:.2f}$' + \
                rf'  $x_3(0) = {x3_0:.2f},$' + \
                rf' $\dot x_3(0) = {x3_dot_0:.2f}$' + \
                rf'  $y_3(0) = {y3_0:.2f},$' + \
                rf' $\dot y_3(0) = {y3_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
    fig.suptitle(overall_title, va='baseline')
    
    # first plot: phi plot 
    ax_a = fig.add_subplot(1,1,1)                  

    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(x1[start : stop], y1[start : stop], 
                axis_labels=r_vs_time_labels, 
                color='blue',
                label='Body 1', 
                title=r'$Two-Body Problem',
                ax=ax_a)
            
    plot_y_vs_x(x2[start : stop], y2[start : stop], 
                axis_labels=r_vs_time_labels, 
                color='red',
                label=r'Body 2', 
                title=r'Two-Body Problem', 
                ax=ax_a)
    
    plot_y_vs_x(x3[start : stop], y3[start : stop], 
                axis_labels=r_vs_time_labels, 
                color='green',
                label=r'Body 3', 
                title=r'Two-Body Problem', 
                ax=ax_a)

# We need control sliders for each variable
slider_G = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$G$ :')
slider_m1 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_1$ :')
slider_m2 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_2$ :')
slider_m3 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_3$ :')
slider_x1_0 = FloatSlider(value=-1, min=-5, max=5, step=0.1,
                      description=r'$x_1(0)$ :')
slider_dx1_0 = FloatSlider(value=0.0, min=-5, max=5., step=0.1,
                      description=r'$\dot x_1(0)$ :')
slider_y1_0 = FloatSlider(value=0, min=-5, max=5, step=0.1,
                      description=r'$y_1(0)$ :')
slider_dy1_0 = FloatSlider(value=0.5, min=-5, max=5., step=0.1,
                      description=r'$\dot y_1(0)$ :')
slider_x2_0 = FloatSlider(value=0.5, min=-5, max=5, step=0.1,
                      description=r'$x_2(0)$ :')
slider_dx2_0 = FloatSlider(value=np.sqrt(3.)/4., min=-5, max=5., step=0.1,
                      description=r'$\dot x_2(0)$ :')
slider_y2_0 = FloatSlider(value=np.sqrt(3.)/2., min=-5, max=5, step=0.1,
                      description=r'$y_2(0)$ :')
slider_dy2_0 = FloatSlider(value=-0.25, min=-5, max=5., step=0.1,
                      description=r'$\dot y_2(0)$ :')
slider_x3_0 = FloatSlider(value=0.5, min=-5, max=5, step=0.1,
                      description=r'$x_3(0)$ :')
slider_dx3_0 = FloatSlider(value=-np.sqrt(3)/4., min=-5, max=5., step=0.1,
                      description=r'$\dot x_3(0)$ :')
slider_y3_0 = FloatSlider(value=-np.sqrt(3.)/2., min=-5, max=5, step=0.1,
                      description=r'$y_3(0)$ :')
slider_dy3_0 = FloatSlider(value=-0.25, min=-5, max=5., step=0.1,
                      description=r'$\dot y_3(0)$ :')

# And now we plot the output with our dictionary of variable:slider correpondences
plot_out = widgets.interactive_output(update_3body_plot,
                                      dict(G=slider_G, m1=slider_m1, m2=slider_m2, m3=slider_m3,
                                           x1_0=slider_x1_0, x2_0=slider_x2_0,
                                           x3_0=slider_x3_0, x1_dot_0=slider_dx1_0,
                                           x2_dot_0=slider_dx2_0, x3_dot_0=slider_dx3_0,
                                           y1_0=slider_y1_0, y2_0=slider_y2_0,
                                           y3_0=slider_y3_0, y1_dot_0=slider_dy1_0,
                                           y2_dot_0=slider_dy2_0, y3_dot_0=slider_dy3_0                                           
                                          )
                                     )

In [ ]:
UI_box = VBox([slider_G, slider_m1, slider_m2, slider_m3,
                slider_x1_0, slider_x2_0, slider_x3_0,
                slider_dx1_0, slider_dx2_0, slider_dx3_0,
                slider_y1_0, slider_y2_0, slider_y3_0,
                slider_dy1_0, slider_dy2_0, slider_dy3_0,
                plot_out])
display(UI_box)

Fancy!

In [ ]:
%%capture

x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, x3, x3_dot, y3, y3_dot = G2.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0, x3_0, x3_dot_0, y3_0, y3_dot_0)

fig_anim1 = plt.figure(figsize=(5,5))
ax_anim1 = fig_anim1.add_subplot(111)
ax_anim1.set_aspect('equal')
ax_anim1.grid()
line_anim1, = ax_anim1.plot(x1[0], y1[0], 'bo')

def animate_double(i):
    xdata = [x1[i], x2[i], x3[i]]
    ydata = [y1[i], y2[i], y3[i]]
    
    line_anim1.set_data(xdata,ydata)
    return line_anim1,

anim1 = animation.FuncAnimation(fig_anim1, animate_double, init_func=None, frames=400, interval=10, blit=True, repeat=False)

In [ ]:
HTML(anim1.to_jshtml())